In [53]:
import numpy as np
import matplotlib.pyplot as plt

import skfuzzy as fuzz
from skfuzzy import control as ctrl
import colorsys

In [4]:
"""
Antecedents HSV
HUE: color represented by number from 0(red) - 360(violet)
{ WARM, COOL }

SATURATION: color saturation represented by number from 0(faded/gray color) - 100(full color)
{ GRAY, VERY_FADED, FADED, SATURATED, VERY_SATURATED }

VALUE: brightness represented by number 0(dark) - 100(light)
{ BLACK, VERY_DARK, DARK, BRIGHT, VERY_BRIGHT }
"""

hue_range = np.arange(0, 361, 1)
hue_fuzzy = ['WARM', 'COOL', 'WARM_']
hue = ctrl.Antecedent(hue_range, 'hue')
hue['WARM'] = fuzz.gaussmf(hue.universe, 0, 60)
hue['COOL'] = fuzz.gaussmf(hue.universe, 180, 60)
hue['WARM_'] = fuzz.gaussmf(hue.universe, 360, 60)


sat = ctrl.Antecedent(np.arange(0, 101, 1), 'saturation')
sat_fuzzy = ['GRAY', 'VERY_FADED', 'FADED', 'SATURATED', 'VERY_SATURATED']
sat['GRAY'] = fuzz.gaussmf(sat.universe, 0, 10)
sat['VERY_FADED'] = fuzz.gaussmf(sat.universe, 25, 10)
sat['FADED'] = fuzz.gaussmf(sat.universe, 50, 10)
sat['SATURATED'] = fuzz.gaussmf(sat.universe, 75, 10)
sat['VERY_SATURATED'] = fuzz.gaussmf(sat.universe, 100, 10)


val = ctrl.Antecedent(np.arange(0, 101, 1), 'value')
val_fuzzy = ['BLACK', 'VERY_DARK', 'DARK', 'BRIGHT', 'VERY_BRIGHT']
val['BLACK'] = fuzz.gaussmf(val.universe, 0, 10)
val['VERY_DARK'] = fuzz.gaussmf(val.universe, 25, 10)
val['DARK'] = fuzz.gaussmf(val.universe, 50, 10)
val['BRIGHT'] = fuzz.gaussmf(val.universe, 75, 10)
val['VERY_BRIGHT'] = fuzz.gaussmf(val.universe, 100, 10)

In [60]:
hue.view()
sat.view()
val.view()

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [58]:
"""
Consequents
TONE: mix of Saturation and Value that indicate if color is neutral or dark/bright
{ NEUTRAL, DARK, BRIGHT }
"""
tone_range = np.arange(0, 12, 1)
tone_fuzzy = ['NEUTRAL', 'DARK', 'BRIGHT']
tone = ctrl.Consequent(tone_range, 'tone')

tone['NEUTRAL'] = fuzz.trapmf(tone.universe, [0, 0, 1, 2])
tone['DARK'] = fuzz.gbellmf(tone.universe, 2, 1, 3)
tone['BRIGHT'] = fuzz.gbellmf(tone.universe, 4, 1, 9.5)
tone['NEUTRAL_'] = fuzz.trimf(tone.universe, [10, 11, 11])

In [57]:
tone.view()


ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

# color identification

In [13]:
"""
Fuzzy rules
for tones
"""
rule1 = ctrl.Rule(val['BLACK'] | sat['GRAY'] | sat['VERY_FADED'], tone['NEUTRAL'], 'Dark colors without color (low brightness/dark) considered neutral')
rule2 = ctrl.Rule(val['VERY_DARK'] & sat['SATURATED'], tone['NEUTRAL'], 'Very dark colors with high saturation')
rule3 = ctrl.Rule(val['DARK'] & sat['FADED'], tone['DARK'], 'Dark color with normal saturation')
rule4 = ctrl.Rule(val['DARK'] & sat['VERY_SATURATED'], tone['BRIGHT'], 'Dark color with high saturation')
rule5 = ctrl.Rule(val['BRIGHT'] & sat['SATURATED'], tone['BRIGHT'], 'Bright color with high saturation')
rule6 = ctrl.Rule(val['VERY_BRIGHT'] & sat['FADED'], tone['BRIGHT'], 'Very bright color with some saturation')
rule7 = ctrl.Rule(val['VERY_BRIGHT'] & sat['VERY_SATURATED'], tone['BRIGHT'], 'Very bright color with high saturation')
rule8 = ctrl.Rule(val['VERY_DARK'] & sat['FADED'], tone['NEUTRAL'], 'Very dark color with faded saturation')

In [14]:
"""
Control system
for tones
"""
tone_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])

In [15]:
"""
Simulation
for tones
"""
tone_sim = ctrl.ControlSystemSimulation(tone_ctrl)

# test
tone_sim.input['saturation'] = 33
tone_sim.input['value'] = 97
tone_sim.compute()
tone_output = tone_sim.output['tone']
print(tone_output)

4.401875643979323


In [ ]:
tone.view(sim=tone_sim)


In [18]:
"""
GetMembership
Returns String representing the Fuzzy value given a variable's range, model, and crisp value
"""
def GetMembership(fuzzy_values, var_range, var_model, crisp_value):
    max_membership = 0
    membership_name = fuzzy_values[0]
    for i in range(len(fuzzy_values)):
        temp_memb = fuzz.interp_membership(var_range, var_model[fuzzy_values[i]].mf, crisp_value)
        if temp_memb > max_membership:
            max_membership = temp_memb
            membership_name = fuzzy_values[i]
    return membership_name

In [19]:
print(GetMembership(tone_fuzzy, tone_range, tone, tone_output))


DARK


# API

In [21]:
"""
Given Saturation and Value, returns a String indicating if the combination 
of both values results in a 'NEUTRAL', 'DARK', or 'BRIGHT' tone.
INPUT:
values - tuple(sat, val)
    + sat - value from 0-100
    + val - value from 0-100
verbose - prints both crisp value and fuzzy value
"""
def GetTone(values, verbose=False):
    tone_sim = ctrl.ControlSystemSimulation(tone_ctrl)
    tone_sim.input['saturation'] = values[0]
    tone_sim.input['value'] = values[1]
    tone_sim.compute()
    tone_output = tone_sim.output['tone']
    tone_membership = GetMembership(tone_fuzzy, tone_range, tone, tone_output)
    if verbose:
        print("TONE CRISP VALUE: ", tone_output)
        print("TONE FUZZY VALUE: ", tone_membership)
    return tone_membership

In [22]:
GetTone((96, 76), verbose=True)


TONE CRISP VALUE:  5.500000000000002
TONE FUZZY VALUE:  BRIGHT


'BRIGHT'

In [24]:
"""
Given Hue, returns a String indicating if the color belongs
to 'WARM' or 'COOL' colors.
INPUT:
hue - value from 0-360
verbose - prints both crisp value and fuzzy value
"""
def GetColorTemp(hue_val, verbose=False):
    temp_membership = GetMembership(hue_fuzzy, hue_range, hue, hue_val)
    if verbose:
        print("TEMP. CRISP VALUE: ", hue_val)
        print("TEMP. FUZZY VALUE: ", temp_membership)
    return temp_membership

In [25]:
GetColorTemp(200, verbose=True)


TEMP. CRISP VALUE:  200
TEMP. FUZZY VALUE:  COOL


'COOL'

In [26]:
"""
Given Hue, Saturation, and Value, returns a String describing
the specified color. The output is composed of both the tone of
the color, and the temperature of the color.
INPUT:
hsv - tuple(hue, sat, val)
    + hue - value from 0-360
    + sat - value from 0-100
    + val - value from 0-100
OUTPUT: (TONE, TEMP) ex. (DARK, WARM)
"""
def GetColorDesc(hsv):
    tone = GetTone((hsv[1], hsv[2]))
    temp = GetColorTemp(hsv[0])
    if temp == "WARM_": temp = "WARM"
    return (tone, temp)

In [27]:
GetColorDesc((57, 72, 75))

('BRIGHT', 'WARM')

In [28]:
"""
Helper function that displays a colored block given a color
represented in HSV format
INPUT:
hsv - tuple(hue, sat, val)
"""
def ShowColor(hsv):
    rgb = colorsys.hsv_to_rgb(hsv[0]/360.0, hsv[1]/100.0, hsv[2]/100.0)
    fig, ax = plt.subplots(nrows=1, ncols=1)
    fig.set_size_inches(3, 2)
    ax.set_facecolor(rgb)
    plt.tick_params(
        left=False,
        bottom=False,
        labelleft=False,
        labelbottom=False)
    plt.title('HSV'+str(hsv))
    plt.show()

In [29]:
ShowColor((200, 10, 100))


ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [30]:
"""
Given Hue, Saturation, and Value, displays a block with the specified HSV color
and prints its color description
INPUT:
hsv - tuple(hue, sat, val)
    + hue - value from 0-360
    + sat - value from 0-100
    + val - value from 0-100
"""
def ShowColorWithDesc(hsv):
    ShowColor(hsv)
    print(GetColorDesc(hsv))

In [31]:
ShowColorWithDesc((372, 70, 50))


ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [32]:
"""
Print colors of a complete outfit in a horizontal bar chart
INPUT:
outfit - tuple(top, bot, shs)
    + top - hsv
    + bot - hsv
    + shs - hsv
"""
def DisplayOutfitColors(outfit, valid_matches=[]):
    colors = [colorsys.hsv_to_rgb(hsv[0]/360.0, hsv[1]/100.0, hsv[2]/100.0) for hsv in outfit]

    plt.rcdefaults()
    fig, ax = plt.subplots()

    clothing = ('TOP', 'BOTTOM', 'SHOES')
    y_pos = np.arange(len(clothing))
    bar_width = [1, 1, 1]

    ax.barh(y_pos, bar_width, align='center', color=colors, height=0.99)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(clothing)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_title('Outfit color scheme')
    ax.set_facecolor((0.8,0.8,0.8))

    plt.tick_params(
            left=True,
            bottom=False,
            labelleft=True,
            labelbottom=False)
    plt.margins(y=0, x=0)

    plt.show()
    
    print("Valid outfit color schemes: ", valid_matches)

In [33]:
top_hsv = (206, 19, 93)
bot_hsv = (30, 21, 71)
shs_hsv = (84, 1, 99)

DisplayOutfitColors((top_hsv, bot_hsv, shs_hsv))

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [34]:
from random import randint


In [35]:
for _ in range(10):
    h = randint(0, 360)
    s = randint(0, 100)
    v = randint(0, 100)
    ShowColorWithDesc((h, s, v))

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [36]:
top_hsv = (206, 19, 93)
bot_hsv = (30, 21, 71)
shs_hsv = (84, 1, 99)
outfit_hsv = (top_hsv, bot_hsv, shs_hsv)

top = GetColorDesc(top_hsv)
bot = GetColorDesc(bot_hsv)
shs = GetColorDesc(shs_hsv)
outfit_desc = (top, bot, shs)

In [37]:
ShowColorWithDesc(top_hsv)
ShowColorWithDesc(bot_hsv)
ShowColorWithDesc(shs_hsv)

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)

In [38]:
"""
Basic outfit follow these rules
- No more than one bright color
- No high contrast between colors (bright warm + dark cool)
- Any number of neutral colors can fit anywhere
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT: 
    True or False
"""
def BasicMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    bright_count = len([i for i in outfit if i[0] == 'BRIGHT'])
    if bright_count > 1: return False
    # Check for high contrast
    
    return True

In [39]:
BasicMatch(outfit_desc)


True

In [40]:
"""
Neutral outfit follow these rules
- Only neutral colors
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT:
    True or False
"""
def NeutralMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    neutral = [color for color in outfit if color[0] == 'NEUTRAL']
    if len(neutral) != len(outfit):
        return False
    
    return True

In [41]:
NeutralMatch(outfit_desc)


True

In [42]:
"""
Analogous outfit follow these rules
- All colors must be within the same temp.
- Any number of neutral colors
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT:
    True or False
"""
def AnalogousMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    cool_count = len([color for color in outfit if color[1] == 'COOL'])
    warm_count = len(outfit) - cool_count
    if cool_count < len(outfit) and warm_count < len(outfit):
        return False
    
    return True

In [43]:
AnalogousMatch(outfit_desc)


False

In [44]:
"""
Contrast outfit follow these rules
- At least one warm color
- Both dark and bright colors present
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT:
    True or False
"""
def ContrastMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    warm_count = len([color for color in outfit if color[1] == 'WARM'])
    if warm_count < 1: return False
    
    dark_count = len([color for color in outfit if color[0] == 'DARK'])
    bright_count = len([color for color in outfit if color[0] == 'BRIGHT'])
    if dark_count < 1 or bright_count < 1:
        return False
    
    return True

In [45]:
ContrastMatch(outfit_desc)


False

In [46]:
"""
Bright summer outfit follow these rules
- At least two warm colors
- At least one bright color
- At most one dark color
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT: 
    True or False
"""
def SummerMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    non_neutral = [color for color in outfit if color[0] != 'NEUTRAL']
    
    warm_count = len([color for color in non_neutral if color[1] == 'WARM'])
    if warm_count < 2: return False
    
    dark_count = len([color for color in non_neutral if color[0] == 'DARK'])
    if dark_count > 1: return False
    
    bright_count = len(non_neutral) - dark_count
    if bright_count < 1: return False
    
    return True

In [47]:
SummerMatch(outfit_desc)


False

In [48]:
"""
Dark winter outfit follow these rules
- At least one dark color
- No bright colors
INPUT:
outfit - tuple(top, bot, shs)
    top - tuple(tone, temp)
    bot - tuple(tone, temp)
    shs - tuple(tone, temp)
OUTPUT:
    True or False
"""
def WinterMatch(outfit):
    top = outfit[0]
    bot = outfit[1] 
    shs = outfit[2]
    
    non_neutral = [color for color in outfit if color[0] != 'NEUTRAL']
    
    dark_count = len([color for color in non_neutral if color[0] == 'DARK'])
    if dark_count < 1: return False
    
    bright_count = len(non_neutral) - dark_count
    if bright_count > 0: return False
    
    return True

In [49]:
WinterMatch(outfit_desc)


False

In [50]:
"""
Iterate outfit over all color schemes and get all valid matches
INPUT:
outfit - tuple(top, bot, shs)
    top - hsv
    bot - hsv
    shs - hsv
OUTPUT:
    All names of valid outfit matches
"""
def GetValidMatches(outfit):
    top = GetColorDesc(outfit[0])
    bot = GetColorDesc(outfit[1])
    shs = GetColorDesc(outfit[2])
    outfit_desc = (top, bot, shs)
    
    rules = {"Basic": BasicMatch, "Neutral": NeutralMatch,
             "Analogous": AnalogousMatch, "Summer": SummerMatch,
            "Winter": WinterMatch}
    valid_matches = []
    for key in rules:
        if rules[key](outfit_desc):
            valid_matches.append(key)
    return valid_matches

In [51]:
"""
male with neutral outfit
top_hsv = (201, 80, 14)
bot_hsv = (210, 18, 39)
shs_hsv = (202, 77, 15)

female with pink shirt
top_hsv = (4, 37, 91)
bot_hsv = (215, 50, 41)
shs_hsv = (2, 26, 47)

woman in winter purple coat
top_hsv = (330, 74, 41)
bot_hsv = (255, 19, 8)
shs_hsv = (0, 0, 15)

male blue shirt
top_hsv = (206, 99, 69)
bot_hsv = (206, 62, 31)
shs_hsv = (211, 12, 97)
-----------------------------------
clown
top_hsv = (162, 51, 67)
bot_hsv = (350, 78, 73)
shs_hsv = (15, 22, 28)

female colorful outfit
top_hsv = (38, 99, 96)
bot_hsv = (360, 72, 73)
shs_hsv = (300, 1, 95)

male with mismatched pink shirt
top_hsv = (343, 44, 100)
bot_hsv = (186, 96, 76)
shs_hsv = (0, 0, 15)
"""

# copy&paste outfit from comments here to test
top_hsv = (330, 74, 41)
bot_hsv = (255, 19, 8)
shs_hsv = (0, 0, 15)
#-----
outfit_hsv = (top_hsv, bot_hsv, shs_hsv)

In [52]:
DisplayOutfitColors(outfit_hsv, valid_matches=GetValidMatches(outfit_hsv))

top = GetColorDesc(top_hsv)
bot = GetColorDesc(bot_hsv)
shs = GetColorDesc(shs_hsv)
outfit_desc = (top, bot, shs)
print(outfit_desc)

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\Cristiana\anaconda3\envs\finalproject\lib\site-packages\matplotlib\backend_bases.py)